In [20]:

import numpy as np
import evaluation.evaluation as ev
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid, GridSearchCV
import pickle

### Load the graphs

In [3]:
#data_list = f_loader.get_dual_data_list(add_structural = True, two_cls = False, force = True) 
#with open('data_list_OCTA.pkl', 'wb') as f:
#    pickle.dump(data_list, f)

with open('data_list_OCTA.pkl', 'rb') as f:
    data_list = pickle.load(f)


data_list_2cls = [] # put CNV and AMD to normal class
data_list_2cls_filt = [] # creates a data_list where 

for data in data_list:
    data.clone
    if data.y > 1:
        data_c = data.clone()
        data_c.y = 0
        data_list_2cls.append(data_c)
    else:
        data_list_2cls_filt.append(data)
        data_list_2cls.append(data)

### Perform some aggregation over the nodes such that we end up with a single feature vector per graph.

In [12]:

# add two features for number of edges and nodes
x_data = np.zeros((len(data_list_2cls), data_list_2cls[0].x.shape[1] + 2))
y_data = np.zeros((len(data_list_2cls),))

# perform mean aggregation (maybe max/min/std would lead to better results)
for i, data in enumerate(data_list_2cls):
    x_data[i,:-2] = data.x.mean(0)
    x_data[i,-2] = data.x.shape[0]
    x_data[i,-1] = data.edge_index.shape[1]
    y_data[i] = data.y


train_mask = np.random.choice(np.arange(0, len(data_list_2cls)), size= int(len(data_list_2cls)*0.8), replace = False)
test_mask = np.delete(np.arange(0, len(data_list_2cls)), train_mask)

# Random Forest

### Grid search for the optimal RF model

In [41]:

param_grid_rf = {'max_depth': [7, 8, 9, 10], 'max_features': ["sqrt", "log2"]}


rf = RandomForestClassifier()
clf_rf = GridSearchCV(rf, param_grid_rf, scoring = "balanced_accuracy")
clf_rf.fit(x_data, y_data)

print(clf_rf.best_score_)
print(clf_rf.best_params_)

rf_best = clf_rf.best_estimator_

0.8741176470588237
{'max_depth': 8, 'max_features': 'sqrt'}


# Support Vector Machine

### Grid search for the optimal support vector machine model

In [42]:
param_grid_svc = {'C': [1, 100, 10000], 'kernel': ["linear", "rbf"]}


svc = SVC()
clf_svc = GridSearchCV(svc, param_grid_svc, scoring = "balanced_accuracy")
clf_svc.fit(x_data, y_data)


print(clf_svc.best_score_)
print(clf_svc.best_params_)

svc_best = clf_svc.best_estimator_


0.8099999999999999
{'C': 100, 'kernel': 'linear'}
